# **Link to the paper**




> DOI	https://doi.org/10.1039/D1NJ04935F



# **Important Chembl links**



1.   blog: https://chembl.blogspot.com/2014/05/a-python-client-for-accessing-chembl.html
2.   webservices: https://chembl.gitbook.io/chembl-interface-documentation/web-services/chembl-data-web-services



In [1]:
import pandas as pd # uses pandas python module to view and analyse data
import requests # this is used to access json files

# **Load dataset**

In [2]:
df = pd.read_excel('/content/Supplementary materials-dataset.xlsx')

In [3]:
df.head()

,Name,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,ES_Count_aasN,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
0,CHEMBL3956266,4.253,158.613,0,2,0,0,0,4,0,...,133.6170,0.53673,103.8950,0.64590,13.8234,81.2150,0.65777,8.93182,458.93,8.44
1,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.73300,11.2985,57.1548,0.68373,7.39849,284.68,6.58
2,CHEMBL3697464,4.128,169.090,0,6,0,0,0,6,0,...,124.9170,0.51461,108.6540,0.57210,13.8269,90.2149,0.60311,10.81810,488.43,8.20
3,CHEMBL3701033,3.330,154.628,0,5,2,0,0,5,0,...,136.9240,0.51344,117.6230,0.65890,14.2145,75.3360,0.55700,9.51509,440.75,8.41
4,CHEMBL3701044,3.425,175.871,0,5,1,0,0,6,0,...,141.5850,0.52205,113.2190,0.69607,15.9872,100.7540,0.65729,9.58809,498.03,8.47


In [ ]:
df.shape

(881, 206)

# **Extract smiles from chembl**

In [ ]:
def get_smiles(chembl_ids):
  smiles_dict = {}
  base_url =   'https://www.ebi.ac.uk/chembl/api/data/molecule/'
  for chembl_id in chembl_ids:
    url = base_url + chembl_id + '.json'
    response = requests.get(url)
    data = response.json()
    smile = data['molecule_structures']['canonical_smiles']
    smiles_dict[chembl_id] = smile
  df_smiles = pd.DataFrame(smiles_dict.items(), columns = ['chembl_id', 'smiles'])
  return df_smiles

In [ ]:
id_smiles = get_smiles(df['Name'].tolist())

In [ ]:
id_smiles.shape

(427, 2)

In [ ]:
# def get_smiles(chembl_ids):
#   smiles_dict = {}
#   for chembl_id in chembl_ids:
#     base_url = 'https://www.ebi.ac.uk/chembl/api/data/molecule/'
#     url = base_url + chembl_id +'.json'
#     response = requests.get(url)
#     data = response.json()
#     smiles_dict[chembl_id] = data['molecule_structures']['canonical_smiles']
#   df_smiles = pd.DataFrame(smiles_dict.items(), columns=['ChEMBL ID', 'SMILES'])
#   return df_smiles

In [ ]:
id_smiles.to_excel('id_smiles.xlsx', index = None)

In [ ]:
import requests

def get_smiles(chembl_ids):
    smiles_dict = {}
    base_url = "https://www.ebi.ac.uk/chembl/api/data/molecule/"

    for chembl_id in chembl_ids:
        url = base_url + chembl_id + ".json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'molecule_structures' in data:
                smiles = data['molecule_structures']['canonical_smiles']
                smiles_dict[chembl_id] = smiles
            else:
                smiles_dict[chembl_id] = "SMILES not found"
        else:
            smiles_dict[chembl_id] = "Error: ChEMBL ID not found or API request failed"
    df_smiles = pd.DataFrame(smiles_dict.items(), columns=['ChEMBL ID', 'SMILES'])
    return smiles_dict


In [ ]:
def get_prop(chembl_ids):
    prop_dict = {}
    base_url = "https://www.ebi.ac.uk/chembl/api/data/molecule/"
    for chembl_id in chembl_ids:
        url = base_url + chembl_id + ".json"
        response = requests.get(url)
        data = response.json()
        smiles = data['molecule_structures']['canonical_smiles']
        prop_dict[chembl_id] = {}
        prop_dict[chembl_id]['smiles'] = smiles
        for key in data['molecule_properties'].keys():
            prop_dict[chembl_id][key] = data['molecule_properties'][key]
    df_prop = pd.DataFrame(prop_dict).T.reset_index(drop=False, names = 'ids')
    return df_prop

In [ ]:
get_prop(df['Name'].tolist()[:5])

,ids,smiles,alogp,aromatic_rings,cx_logd,cx_logp,cx_most_apka,cx_most_bpka,full_molformula,full_mwt,...,molecular_species,mw_freebase,mw_monoisotopic,np_likeness_score,num_lipinski_ro5_violations,num_ro5_violations,psa,qed_weighted,ro3_pass,rtb
0,CHEMBL3956266,CCN(c1cc(C#CC(C)(C)O)cc(C(=O)NCc2c(C)cc(C)[nH]...,4.05,2,0.91,3.50,11.65,10.20,C31H44N4O3,520.72,...,BASE,520.72,520.3413,-0.98,1,1,88.67,0.48,N,7
1,CHEMBL418052,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O...,-1.44,2,-4.03,-4.02,1.81,9.50,C14H20N6O5S,384.42,...,ZWITTERION,384.42,384.1216,0.83,2,1,182.63,0.35,N,7
2,CHEMBL3697464,CCN(c1cc(-c2ccc(CN3CC[C@@H](F)C3)cc2)cc(C(=O)N...,5.45,3,3.26,3.82,11.64,7.82,C34H43FN4O3,574.74,...,NEUTRAL,574.74,574.3319,-1.21,2,2,77.67,0.36,N,9
3,CHEMBL3701033,CCN(c1cc(-c2cnccn2)cc(C(=O)NCc2c(C)cc(C)[nH]c2...,4.39,3,0.11,2.70,11.65,10.20,C30H40N6O2,516.69,...,BASE,516.69,516.3213,-1.28,1,1,94.22,0.46,N,8
4,CHEMBL3701044,CCN(c1cc(-c2ccc(N3CCNCC3)nc2)cc(C(=O)NCc2c(C(C...,4.52,3,2.01,3.41,11.63,8.79,C34H46N6O3,586.78,...,BASE,586.78,586.3631,-1.25,1,1,102.59,0.34,N,9


# **Truncate the dataset**

In [ ]:
df.shape

(881, 206)

In [ ]:
df['Name'].value_counts()

CHEMBL418052     22
CHEMBL3414621     9
CHEMBL3264787     6
CHEMBL3701037     6
CHEMBL3701005     6
                 ..
CHEMBL3770973     1
CHEMBL4080043     1
CHEMBL4539350     1
CHEMBL3697416     1
CHEMBL4483642     1
Name: Name, Length: 427, dtype: int64

In [ ]:
df['Name'].nunique()

427

In [ ]:
df[df['Name']=='CHEMBL418052'].head()

,Name,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,ES_Count_aasN,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
1,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.733,11.2985,57.1548,0.68373,7.39849,284.68,6.58
86,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.733,11.2985,57.1548,0.68373,7.39849,284.68,6.33
152,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.733,11.2985,57.1548,0.68373,7.39849,284.68,6.33
185,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.733,11.2985,57.1548,0.68373,7.39849,284.68,6.55
213,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.733,11.2985,57.1548,0.68373,7.39849,284.68,5.16


In [ ]:
(df[df['Name']=='CHEMBL418052'].iloc[1,:] == df[df['Name']=='CHEMBL418052'].iloc[2,:]).value_counts()

True    206
dtype: int64

In [ ]:
df[df['Name']=='CHEMBL418052']['pChEMBL Value'].value_counts()

6.58    3
6.33    3
6.55    3
5.32    3
6.42    3
5.16    1
5.21    1
4.19    1
4.96    1
5.23    1
5.72    1
4.78    1
Name: pChEMBL Value, dtype: int64

In [ ]:
id_smiles.shape

(427, 2)

In [4]:
df_trun = df.drop_duplicates(subset=['Name'], keep='first')

In [5]:
df_trun.head()

,Name,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,ES_Count_aasN,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
0,CHEMBL3956266,4.253,158.613,0,2,0,0,0,4,0,...,133.6170,0.53673,103.8950,0.64590,13.8234,81.2150,0.65777,8.93182,458.93,8.44
1,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.73300,11.2985,57.1548,0.68373,7.39849,284.68,6.58
2,CHEMBL3697464,4.128,169.090,0,6,0,0,0,6,0,...,124.9170,0.51461,108.6540,0.57210,13.8269,90.2149,0.60311,10.81810,488.43,8.20
3,CHEMBL3701033,3.330,154.628,0,5,2,0,0,5,0,...,136.9240,0.51344,117.6230,0.65890,14.2145,75.3360,0.55700,9.51509,440.75,8.41
4,CHEMBL3701044,3.425,175.871,0,5,1,0,0,6,0,...,141.5850,0.52205,113.2190,0.69607,15.9872,100.7540,0.65729,9.58809,498.03,8.47


In [6]:
df_trun.shape

(427, 206)

In [7]:
df_trun_ = df.drop_duplicates(subset=['Name'], keep=False)
df_trun_.shape

(177, 206)

# **Build Model**

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [8]:
df_trun.head()

,Name,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,ES_Count_aasN,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
0,CHEMBL3956266,4.253,158.613,0,2,0,0,0,4,0,...,133.6170,0.53673,103.8950,0.64590,13.8234,81.2150,0.65777,8.93182,458.93,8.44
1,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.73300,11.2985,57.1548,0.68373,7.39849,284.68,6.58
2,CHEMBL3697464,4.128,169.090,0,6,0,0,0,6,0,...,124.9170,0.51461,108.6540,0.57210,13.8269,90.2149,0.60311,10.81810,488.43,8.20
3,CHEMBL3701033,3.330,154.628,0,5,2,0,0,5,0,...,136.9240,0.51344,117.6230,0.65890,14.2145,75.3360,0.55700,9.51509,440.75,8.41
4,CHEMBL3701044,3.425,175.871,0,5,1,0,0,6,0,...,141.5850,0.52205,113.2190,0.69607,15.9872,100.7540,0.65729,9.58809,498.03,8.47


In [19]:
y = df_trun['pChEMBL Value']
X = df_trun.iloc[:, 1:-1]

In [21]:
y.shape

(427,)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
train_r2 = rf.score(X_train, y_train)
test_r2 = rf.score(X_test, y_test)

In [24]:
print(train_r2, test_r2)

0.9473466572199505 0.42550714379358556


In [25]:
import random
scores = []
for i in range(10):
  score = []
  i = random.randint(100,1000)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=i)
  rf = RandomForestRegressor()
  rf.fit(X_train, y_train)
  train_r2 = rf.score(X_train, y_train)
  test_r2 = rf.score(X_test, y_test)
  score += [i] + [train_r2] + [test_r2]
  scores.append(score)
results = pd.DataFrame(scores, columns = ['random_state', 'train_r2', 'test_r2'])


In [27]:
results.sort_values('test_r2', ascending=False)

,random_state,train_r2,test_r2
4,286,0.941682,0.654966
2,174,0.944223,0.637191
0,995,0.941906,0.629908
3,289,0.941470,0.618251
8,536,0.941807,0.601377
1,408,0.944218,0.588104
5,413,0.942344,0.580829
9,683,0.944655,0.580228
6,852,0.942995,0.486841
7,305,0.946830,0.467455


In [32]:
id_smiles.head()

,Name,SMILES
0,CHEMBL3956266,CCN(c1cc(C#CC(C)(C)O)cc(C(=O)NCc2c(C)cc(C)[nH]...
1,CHEMBL418052,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O...
2,CHEMBL3697464,CCN(c1cc(-c2ccc(CN3CC[C@@H](F)C3)cc2)cc(C(=O)N...
3,CHEMBL3701033,CCN(c1cc(-c2cnccn2)cc(C(=O)NCc2c(C)cc(C)[nH]c2...
4,CHEMBL3701044,CCN(c1cc(-c2ccc(N3CCNCC3)nc2)cc(C(=O)NCc2c(C(C...


In [31]:
id_smiles.rename(columns={'ChEMBL ID': 'Name'}, inplace=True)

In [30]:
df_trun.head()

,Name,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,ES_Count_aasN,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
0,CHEMBL3956266,4.253,158.613,0,2,0,0,0,4,0,...,133.6170,0.53673,103.8950,0.64590,13.8234,81.2150,0.65777,8.93182,458.93,8.44
1,CHEMBL418052,-4.356,87.128,2,2,4,0,0,1,0,...,87.4699,0.58665,71.5657,0.73300,11.2985,57.1548,0.68373,7.39849,284.68,6.58
2,CHEMBL3697464,4.128,169.090,0,6,0,0,0,6,0,...,124.9170,0.51461,108.6540,0.57210,13.8269,90.2149,0.60311,10.81810,488.43,8.20
3,CHEMBL3701033,3.330,154.628,0,5,2,0,0,5,0,...,136.9240,0.51344,117.6230,0.65890,14.2145,75.3360,0.55700,9.51509,440.75,8.41
4,CHEMBL3701044,3.425,175.871,0,5,1,0,0,6,0,...,141.5850,0.52205,113.2190,0.69607,15.9872,100.7540,0.65729,9.58809,498.03,8.47


In [33]:
df_merged = id_smiles.merge(df_trun, how='inner', on=['Name'])

In [34]:
df_merged.head()

,Name,SMILES,ALogP,ALogP_MR,ES_Count_aaaC,ES_Count_aaCH,ES_Count_aaN,ES_Count_aaNH,ES_Count_aaO,ES_Count_aasC,...,Shadow_XY,Shadow_XYfrac,Shadow_XZ,Shadow_XZfrac,Shadow_Ylength,Shadow_YZ,Shadow_YZfrac,Shadow_Zlength,Molecular_Volume,pChEMBL Value
0,CHEMBL3956266,CCN(c1cc(C#CC(C)(C)O)cc(C(=O)NCc2c(C)cc(C)[nH]...,4.253,158.613,0,2,0,0,0,4,...,133.6170,0.53673,103.8950,0.64590,13.8234,81.2150,0.65777,8.93182,458.93,8.44
1,CHEMBL418052,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O...,-4.356,87.128,2,2,4,0,0,1,...,87.4699,0.58665,71.5657,0.73300,11.2985,57.1548,0.68373,7.39849,284.68,6.58
2,CHEMBL3697464,CCN(c1cc(-c2ccc(CN3CC[C@@H](F)C3)cc2)cc(C(=O)N...,4.128,169.090,0,6,0,0,0,6,...,124.9170,0.51461,108.6540,0.57210,13.8269,90.2149,0.60311,10.81810,488.43,8.20
3,CHEMBL3701033,CCN(c1cc(-c2cnccn2)cc(C(=O)NCc2c(C)cc(C)[nH]c2...,3.330,154.628,0,5,2,0,0,5,...,136.9240,0.51344,117.6230,0.65890,14.2145,75.3360,0.55700,9.51509,440.75,8.41
4,CHEMBL3701044,CCN(c1cc(-c2ccc(N3CCNCC3)nc2)cc(C(=O)NCc2c(C(C...,3.425,175.871,0,5,1,0,0,6,...,141.5850,0.52205,113.2190,0.69607,15.9872,100.7540,0.65729,9.58809,498.03,8.47


In [35]:
df_merged.shape

(427, 207)

In [36]:
df_merged.nunique()

Name                427
SMILES              427
ALogP               372
ALogP_MR            383
ES_Count_aaaC         2
                   ... 
Shadow_YZ           427
Shadow_YZfrac       414
Shadow_Zlength      427
Molecular_Volume    327
pChEMBL Value       249
Length: 207, dtype: int64